# **차량 공유업체의 차량 파손 여부 분류하기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### (1) 데이터셋 불러오기 
- **세부요구사항**
    - Car_Images.zip 파일을 C:/Datasets/ 경로에 압축 해제합니다.
    - zipfile 모듈을 이용하거나 다른 방식을 사용해도 됩니다.
        - 참고 자료 : [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 폴더구조(로컬)
        * C:/Datasets/ : 압축파일
        * C:/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 폴더구조(구글드라이브브)
        * /content/drive/MyDrive/Datasets/ : 압축파일
        * /content/drive/MyDrive/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 압축을 해제하면 다음과 같은 두 하위 폴더가 생성됩니다.
        * normal, abnormal : 각 폴더에는 이미지들이 있습니다.
        * 이후 단계에서 해당 경로로 부터 validation, test 셋을 추출하게 됩니다.
        

In [ ]:
import zipfile

In [ ]:
# 압축파일 경로
# 구글 드라이브인 경우 경로에 맞게 지정하세요.
dataset_path  = '/content/drive/MyDrive/Datasets/'

file_path = dataset_path + 'Car_Images.zip'

In [ ]:
# 압축 해제
data = zipfile.ZipFile(file_path)
data.extractall('/content/drive/MyDrive/Datasets/Car_Images_train/')

### (2) 이미지 저장을 위한 폴더 생성
- **세부요구사항**
    - train, validation, test 을 위해 각각 하위 폴더 normal과 abnormal를 준비합니다.
        - train
            * 정상 이미지 저장소 : C:/Datasets/Car_Images_train/normal/ 
                * 구글드라이브 :   /content/drive/MyDrive/Datasets/Car_Images_train/normal/
            * 파손 이미지 저장소 : C:/Datasets/Car_Images_train/abnormal/
                * 구글드라이브 : /content/drive/MyDrive/Datasets/Car_Images_train/abnormal/
        - val, test 역시 동일한 구조로 생성합니다.
    - 직접 탐색기에서 폴더를 생성할 수도 있고, os 모듈을 이용하여 코드로 작성할 수도 있습니다.
        - 참고 자료 : [os document](https://docs.python.org/3/library/os.html)

In [ ]:
import os

# train 폴더는 압축을 해제하면서 이미 생성 되어 있습니다.

# test 폴더 만들기 os.mkdir()
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_test/')
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_test/normal/')
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_test/abnormal/')

# validation 폴더 만들기
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_val/')
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_val/normal/')
os.mkdir('/content/drive/MyDrive/Datasets/Car_Images_val/abnormal/')


## 2.데이터 전처리

### (1) 데이터 분할 : Training set | Validation set | Test set 생성
- **세부요구사항**
    - Training set, Validation set, Test set을 만듭니다.
        * size
            * test : 전체에서 20%를 추출합니다.
            * validation : test를 떼어낸 나머지에서 다시 20%를 추출합니다.
        * 데이터는 랜덤하게 추출해야 합니다.
            - random, shutil 모듈을 이용하여 랜덤하게 추출할 수 있습니다.
                - [random document](https://docs.python.org/3/library/random.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
            * 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

#### 1) test, validation 크기를 지정

In [ ]:
tr_n_path = '/content/drive/MyDrive/Datasets/Car_Images_train/normal/'
tr_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_train/abnormal/'

test_n_path = '/content/drive/MyDrive/Datasets/Car_Images_test/normal/'
test_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_test/abnormal/'

val_n_path = '/content/drive/MyDrive/Datasets/Car_Images_val/normal/'
val_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_val/abnormal/'

In [ ]:
import random, shutil

In [ ]:
# 전체 이미지 갯수를 확인합니다.
len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path))

In [ ]:
# test 사이즈 : 전체 이미지의 20%
te_data_num = [round(len(os.listdir(tr_n_path))*0.2), round(len(os.listdir(tr_ab_path))*0.2)]
print(te_data_num)

# validation 사이즈 : test를 제외한 나머지 중에서 20%
val_data_num = [ round((len(os.listdir(tr_n_path))-te_data_num[0])*0.2) , round((len(os.listdir(tr_n_path))-te_data_num[1])*0.2) ]
print(val_data_num)

# train 사이즈
train_data_num = [len(os.listdir(tr_n_path)) - te_data_num[0] - val_data_num[0],
                  len(os.listdir(tr_ab_path))- te_data_num[1] - val_data_num[1]]
print(train_data_num)

#### 2) test 셋 추출

In [ ]:
random.seed(2023)

# Car_Images_test/normal
for i in range(0, te_data_num[0]):
    src = random.choice(os.listdir(tr_n_path))
    shutil.move(tr_n_path + src, test_n_path + src)

# Car_Images_test/abnormal
for i in range(0, te_data_num[1]):
    src = random.choice(os.listdir(tr_ab_path))
    shutil.move(tr_ab_path + src, test_ab_path + src)


In [ ]:
# 추출 후 이미지 갯수 확인
print(len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path)))
print(len(os.listdir(test_n_path)) , len(os.listdir(test_ab_path)))
print(len(os.listdir(val_n_path)) , len(os.listdir(val_ab_path)))

#### 3) validation 셋 추출

In [ ]:
random.seed(2023)

# Car_Images_val/normal
for i in range(0, val_data_num[0]):
    src = random.choice(os.listdir(tr_n_path))
    shutil.move(tr_n_path + src, val_n_path + src)

# Car_Images_val/abnormal
for i in range(0, val_data_num[1]):
    src = random.choice(os.listdir(tr_ab_path))
    shutil.move(tr_ab_path + src, val_ab_path + src)


In [ ]:
# 추출 후 이미지 갯수 확인
print(len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path)))
print(len(os.listdir(test_n_path)) , len(os.listdir(test_ab_path)))
print(len(os.listdir(val_n_path)) , len(os.listdir(val_ab_path)))

### (2) 데이터 복사 및 이동
- **세부요구사항**
    - 분할된 데이터를 복사 이동합니다.
        - 새로운 폴더에 저장하는 데이터로 "3.모델링I"에서 사용합니다.
        - 기존 폴더는 "4.모델링II > (1) Data Augmentation"에서 사용합니다.
    - Training set | Validation set | Test set의 데이터를 **새로운 폴더**에 복사하세요.
        - 새로운 폴더 명
            * copy_images/trainset
            * copy_images/validset
            * copy_images/testset
        - 새로운 폴더에는 normal, abnormal 파일 모두를 복사합니다. 
            * 파일을 구분하기 위해 abnormal 파일들은 파일명 앞에 접두사 'ab_'를 붙입시다.
        - os, shutil 모듈을 활용하세요.

#### 1) abnormal 파일 복사

In [ ]:
# 폴더 생성
copy_path = '/content/drive/MyDrive/Datasets/copy_images/'
os.mkdir(copy_path)
os.mkdir(copy_path + 'trainset/')
os.mkdir(copy_path + 'validset/')
os.mkdir(copy_path + 'testset/')

* 복사하기 : shutil.copytree()

In [ ]:
from distutils.dir_util import copy_tree

copy_tree(tr_ab_path , copy_path + 'trainset/')
copy_tree(val_ab_path , copy_path + 'validset/')
copy_tree(test_ab_path , copy_path + 'testset/')

* abnormal 이미지 이름의 접두어 "ab_" 붙이기 : os.rename

In [ ]:
for fname in os.listdir(copy_path + 'trainset/'):
    os.rename(copy_path + 'trainset/' + fname, copy_path + 'trainset/' + 'ab_' + fname)

for fname in os.listdir(copy_path + 'validset/'):
    os.rename(copy_path + 'validset/' + fname, copy_path + 'validset/' + 'ab_' + fname)

for fname in os.listdir(copy_path + 'testset/'):
    os.rename(copy_path + 'testset/' + fname, copy_path + 'testset/' + 'ab_' + fname)

#### 2) normal 파일 복사

In [ ]:
copy_tree(tr_n_path , copy_path + 'trainset/')
copy_tree(val_n_path , copy_path + 'validset/')
copy_tree(test_n_path , copy_path + 'testset/')

* 데이터 갯수 조회

In [ ]:
print(len(os.listdir(dataset_path+'copy_images/trainset/')))
print(len(os.listdir(dataset_path+'copy_images/validset/')))
print(len(os.listdir(dataset_path+'copy_images/testset/')))